In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import pickle
import numpy as np
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient


class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]


In [2]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()


train loss:2.300232698123317
=== epoch:1, train acc:0.146, test acc:0.159 ===
train loss:2.297554492615374
train loss:2.2934089717622625
train loss:2.291361205428947
train loss:2.2829570762471194
train loss:2.271632120460171
train loss:2.258565966669403
train loss:2.2584883225939962
train loss:2.2296711909767013
train loss:2.2149255297288466
train loss:2.1924826640668322
train loss:2.1539180208554067
train loss:2.113744161396101
train loss:2.0399499855729464
train loss:2.017197449849174
train loss:1.9828784777292268
train loss:1.8589670193089425
train loss:1.8134403372276886
train loss:1.743645673613714
train loss:1.6812120817300755
train loss:1.6455265905501673
train loss:1.516445077806069
train loss:1.3671100046355986
train loss:1.4060069813997789
train loss:1.2962260524926297
train loss:1.290786504487861
train loss:1.1512815364460574
train loss:1.0736150070857633
train loss:1.04927451082066
train loss:0.8658035501773726
train loss:0.8861300851315324
train loss:1.0505337759395053
tra

train loss:0.2686799037473335
train loss:0.20564633046744887
train loss:0.283332910376981
train loss:0.12147157367332842
train loss:0.27952017517116756
train loss:0.264541531027073
train loss:0.2239201397573656
train loss:0.17521578320587225
train loss:0.14305272578073858
train loss:0.22768400928285384
train loss:0.16243195478223818
train loss:0.2217008287762305
train loss:0.2630038848959219
train loss:0.19134893878869136
train loss:0.19071450149905453
train loss:0.24692829305875288
train loss:0.322844258885445
train loss:0.1972847273986322
train loss:0.225952282742698
train loss:0.12107633359354891
train loss:0.18637664374970822
train loss:0.10448031709997788
train loss:0.3354066768798758
train loss:0.28399999976891727
train loss:0.1467241515822312
train loss:0.31494676191568177
train loss:0.19726139423386432
train loss:0.16969725086558735
train loss:0.11264194562182193
train loss:0.22332985188194565
train loss:0.20836079136472094
train loss:0.2301040915631571
train loss:0.15681565736

train loss:0.2602471572594343
train loss:0.13939138920984914
train loss:0.07934522326190635
train loss:0.11754085229723969
train loss:0.10113797297205233
train loss:0.2241230129592775
train loss:0.1467490683139956
train loss:0.1511636885020571
train loss:0.19468093093141964
train loss:0.08706590970234815
train loss:0.1272634140282028
train loss:0.0843983426855777
train loss:0.150787720370527
train loss:0.08500957798304624
train loss:0.09978136308144483
train loss:0.1716677007740455
train loss:0.09985802975280789
train loss:0.19329474149688378
train loss:0.10004372577345164
train loss:0.17301792594591978
train loss:0.2655771610779279
train loss:0.0642659087598737
train loss:0.08401812869035273
train loss:0.14983791501482357
train loss:0.09213535902438112
train loss:0.1681732692324524
train loss:0.09858291547382392
train loss:0.11934754198255883
train loss:0.06456889608754532
train loss:0.12110782280668404
train loss:0.10718458074615234
train loss:0.08968641211428176
train loss:0.1276375

train loss:0.0933966213087431
train loss:0.04870790593386342
train loss:0.11868108338903328
train loss:0.07787571184191189
train loss:0.069386965361459
train loss:0.10693032650176595
train loss:0.09869338836804072
train loss:0.07100994945494353
train loss:0.040872051590296665
train loss:0.06220237088725906
train loss:0.17762708370491673
train loss:0.03926793393916355
train loss:0.09009827509767726
train loss:0.1218288643577081
train loss:0.05978246017816008
train loss:0.149442503695316
train loss:0.11101159287423913
train loss:0.13267978268152897
train loss:0.05634022616846987
train loss:0.04667651867987299
train loss:0.10693419095928286
train loss:0.12709631947477829
train loss:0.06619098833620375
train loss:0.09123492713766529
train loss:0.07786463312618723
train loss:0.030666197732635733
train loss:0.0635054441525607
train loss:0.03709616463830379
train loss:0.18177483433092892
train loss:0.09889283402453568
train loss:0.07046142145502621
train loss:0.17570916776411988
train loss:0.

train loss:0.05776000206718551
train loss:0.1300859545413486
train loss:0.05013227064596988
train loss:0.19001766022647007
train loss:0.06451332025548577
train loss:0.14039384916377354
train loss:0.1542080661560701
train loss:0.08178242753570437
train loss:0.055795073630832574
train loss:0.06532171777030121
train loss:0.029028847054342593
train loss:0.05412415242800714
train loss:0.054785871043557186
train loss:0.061083624139227036
train loss:0.07813307923303002
train loss:0.10969014505431582
train loss:0.08150360501214828
train loss:0.04019537996955119
train loss:0.13789062760727083
train loss:0.09663171836923809
train loss:0.11224620540339915
train loss:0.08724655816481836
train loss:0.053313823265973836
train loss:0.04909398250989125
train loss:0.05234416890916365
train loss:0.12604923858233819
train loss:0.062469501550303466
train loss:0.07003864751334893
train loss:0.04076956244860543
train loss:0.08934885266085908
train loss:0.040147259754846384
train loss:0.06600646639665764
tra

...

train loss:9.62238526943244e-05
train loss:0.002575279407784593
train loss:0.0021340100442210448
train loss:0.0003316324508680433
train loss:6.630205618127979e-05
train loss:0.0016851689057224206
train loss:0.0004810035293610658
train loss:0.000169687392448978
train loss:0.0026343836332349217
train loss:0.00015507430448758147
train loss:0.0013759002272241082
train loss:6.613476260408034e-05
train loss:0.0018839124919757784
train loss:0.002132568069647289
train loss:0.001393949138371228
train loss:0.0001328225451436596
train loss:0.00010644716076798521
train loss:0.0009493735588586436
train loss:0.0004756880287046274
train loss:1.3249354969224758e-05
train loss:2.8819504297229616e-05
train loss:0.0016662913213920238
train loss:0.004192840450235668
train loss:0.002089414559260205
train loss:0.002432703034335821
train loss:0.0015244498215207236
train loss:0.0007360841855379747
train loss:0.004437576398516221
train loss:0.0007595629362427245
train loss:0.0013838795331892976
train loss:0.00

train loss:0.0026119070395359627
train loss:0.0008421871817123433
train loss:4.5931459863516276e-05
train loss:0.0005762698758828406
train loss:0.0005326774595019313
train loss:0.00012002220253553128
train loss:0.0027559053484343425
train loss:0.001019367982271273
train loss:0.011096197442241653
train loss:0.00012586768644642292
train loss:8.779879824508062e-05
train loss:0.00040531880924672546
train loss:1.8931170066445493e-05
train loss:0.000401403463776582
train loss:0.00228733085172311
train loss:0.000821429730785214
train loss:0.0004359645594064919
train loss:0.00015187862359048002
train loss:0.0015701939467921847
train loss:6.912754463133175e-05
train loss:0.005755980254853992
train loss:0.0002469207019203887
train loss:0.0005192401721466722
train loss:7.151308633742351e-06
train loss:0.005970443624498521
train loss:0.0007241112419446364
train loss:0.0008548201506195776
train loss:0.0007572344111267406
train loss:0.0005397641708592388
train loss:0.0030796366602217086
train loss:0

train loss:6.19258023493135e-05
train loss:0.001181716173502476
train loss:0.0009701340826898243
train loss:0.010308878366414637
train loss:0.00013102304093795422
train loss:1.250761830797865e-05
train loss:0.0010056496157016738
train loss:0.0023933709593287125
train loss:7.43057821159123e-05
train loss:0.0009451404314599952
train loss:0.001646677466209697
train loss:0.0005334536028528399
train loss:0.0003827314236051066
train loss:0.0034914532916704654
train loss:4.630250031030276e-05
train loss:0.0002631891938017377
train loss:0.0016861271687405486
train loss:0.0003498405704423038
train loss:0.0003281111057103662
train loss:0.00037079456898851426
train loss:0.00211541178576233
train loss:0.001449666617538013
train loss:9.627551432121136e-06
train loss:0.0007993945218000873
train loss:0.0008268058718138593
train loss:0.0016090953712353331
train loss:0.00018053946302534467
train loss:0.00035552761511362894
train loss:0.0009040811078898963
train loss:8.892080908163252e-05
train loss:0.0

train loss:0.00039299948588438085
train loss:0.00113592518780449
train loss:0.0017076098656594269
train loss:0.0031583459568415755
train loss:0.0007648035612067903
train loss:1.2886333863030571e-05
train loss:0.0002136737711393995
train loss:0.002456515730722854
train loss:0.0015634255406202857
=============== Final Test Accuracy ===============
test acc:0.9896
Saved Network Parameters!


<Figure size 640x480 with 1 Axes>